In [1]:
# Input
import re
parser_init = re.compile(r"initial state: ([#.]+)")
parser_rule = re.compile(r"([#.]{5}) => ([#.])")

def interpret(s):
    return [c == '#' for c in s]

def bool_vec_to_int(v):
    i = 1
    code = 0
    for b in v:
        if b:
            code += i
        i *= 2
    return code

with open("Input/12.txt") as file:
    initial_state = interpret(parser_init.match(file.readline()).group(1))
    file.readline()
    rules = []
    for line in file:
        prev, post = parser_rule.match(line).groups()
        prev, post = bool_vec_to_int(interpret(prev)), (post == '#')
        rules.append((prev, post))
    rules = dict(rules)
    rules = [rules[i] for i in range(len(rules))]
        
class Pots:
    def __init__(self, initial_state, rules):
        self.state = initial_state
        self.rules = rules
        self.offset = 0
        
    def __str__(self):
        result = ""
        for b in self.state:
            if b:
                result += '#'
            else:
                result += '.'
        return result
    
    def trim(self):
        while self.state and not self.state[0]:
            self.state.pop(0)
            self.offset += 1
        while self.state and not self.state[-1]:
            self.state.pop()
    
    def rule(self, x):
        i = []
        i.append(x >= 2 and self.state[x-2])
        i.append(x >= 1 and self.state[x-1])
        i.append(self.state[x])
        i.append(x < len(self.state) - 1 and self.state[x+1])
        i.append(x < len(self.state) - 2 and self.state[x+2])
        return bool_vec_to_int(i)

    def step(self):
        self.state = [False, False] + self.state + [False, False]
        self.offset -= 2
        self.state = [rules[self.rule(x)] for x in range(len(self.state))]
        self.trim()
        
    def score(self):
        result = 0
        for x in range(len(self.state)):
            if self.state[x]:
                result += x + self.offset
        return result

In [2]:
# Part 1
pots = Pots(initial_state, rules)
for _ in range(20):
    pots.step()
print(pots.score())

3061


In [3]:
# Part 2
from collections import deque

pots = Pots(initial_state, rules)
buffer = deque([0 for _ in range(1000)])
prev = pots.score()
for i in range(50000000000):
    pots.step()
    buffer.popleft()
    cur = pots.score()
    buffer.append(cur - prev)
    prev = cur
    step = buffer[0]
    if all([b == step for b in buffer]):
        gens = i + 1
        break

final_score = pots.score() + (50000000000 - gens) * step
print(final_score)

4049999998575
